In [ ]:
!pip install torch torchvision torchaudio lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [ ]:
# token_to_id = {'what': 0,
#                'is': 1,
#                'statquest': 2,
#                'awesome': 3,
#                '<EOS>': 4,
#                }
# id_to_token = dict(map(reversed, token_to_id.items()))

In [ ]:
token_to_id = {'who': 0,
               'is': 1,
               'touati': 2,
               'kamel': 3,
               '<EOS>': 4,
               "i'm": 5,
               'estin': 6,
               'student': 7,
               }

id_to_token = {v: k for k, v in token_to_id.items()}

In [ ]:
# inputs = torch.tensor([[token_to_id["what"],
#                        token_to_id["is"],
#                        token_to_id["statquest"],
#                        token_to_id["<EOS>"],
#                        token_to_id["awesome"]],
#                       [token_to_id["statquest"],
#                        token_to_id["is"],
#                        token_to_id["what"],
#                        token_to_id["<EOS>"],
#                        token_to_id["awesome"]]])

# labels = torch.tensor([[token_to_id["is"],
#                         token_to_id["statquest"],
#                         token_to_id["awesome"],
#                         token_to_id["<EOS>"],
#                         token_to_id["what"]],
#                        [token_to_id["is"],
#                         token_to_id["what"],
#                         token_to_id["<EOS>"],
#                         token_to_id["awesome"],
#                         token_to_id["<EOS>"]]])

# dataset = TensorDataset(inputs, labels)
# dataloader = DataLoader(dataset)

In [ ]:
inputs = torch.tensor([[token_to_id["who"],
                        token_to_id["is"],
                        token_to_id["touati"],
                        token_to_id["kamel"],
                        token_to_id["<EOS>"],
                        token_to_id["i'm"],
                        token_to_id["estin"],
                        token_to_id["student"]]
                       ,[token_to_id["touati"],
                        token_to_id["kamel"],
                        token_to_id["is"],
                        token_to_id["who"],
                        token_to_id["<EOS>"],
                        token_to_id["i'm"],
                        token_to_id["estin"],
                        token_to_id["student"]]
                       ])

labels = torch.tensor([[token_to_id["is"],
                        token_to_id["touati"],
                        token_to_id["kamel"],
                        token_to_id["i'm"],
                        token_to_id["i'm"],
                        token_to_id["estin"],
                        token_to_id["student"],
                        token_to_id["<EOS>"],
                        ],
                       [
                        token_to_id["kamel"],
                        token_to_id["is"],
                        token_to_id["who"],
                        token_to_id["i'm"],
                        token_to_id["i'm"],
                        token_to_id["estin"],
                        token_to_id["student"],
                        token_to_id["<EOS>"],
                        ]])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [ ]:
class PositionEncoding(nn.Module):

  def __init__(self, d_model=2, max_len=10):
    super().__init__()

    pe = torch.zeros(max_len, d_model)

    position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
    embedding_index = torch.arange(start=0, end=d_model, step=2).float()
    div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    self.register_buffer('pe', pe)

  def forward(self, word_embeddings):
    return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [ ]:
class Attention(nn.Module):
  def __init__(self, d_model=2):
    super().__init__()

    self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
    self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
    self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

    self.row_dim = 0
    self.col_dim = 1

  def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
      q = self.W_q(encodings_for_q)
      k = self.W_k(encodings_for_k)
      v = self.W_v(encodings_for_v)

      sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))
      scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)
      if mask is not None:
        scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
      attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
      attention_scores = torch.matmul(attention_percents, v)

      return attention_scores

In [ ]:
class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self, num_tokens=4, d_model=2, max_len=10):
      super().__init__()

      self.we = nn.Embedding(num_embeddings=num_tokens,
                            embedding_dim=d_model)
      self.pe = PositionEncoding(d_model=d_model,
                                max_len=max_len)
      self.self_attention = Attention(d_model=d_model)
      self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

      self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):

      word_embeddings = self.we(token_ids)
      position_encoded = self.pe(word_embeddings)

      # mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
      mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0)), device=token_ids.device))
      mask = mask == 0

      self_attention_values = self.self_attention(position_encoded,
                                                  position_encoded,
                                                  position_encoded,
                                                  mask=mask)

      residual_connection_values = position_encoded + self_attention_values
      fc_layer_output = self.fc_layer(residual_connection_values)
      return fc_layer_output

    def configure_optimizers(self):
      return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
      input_tokens, labels = batch
      output = self.forward(input_tokens[0])
      loss = self.loss(output, labels[0])
      return loss

In [ ]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=10)

# model_input = torch.tensor([token_to_id["what"],
#                            token_to_id["is"],
#                            token_to_id["statquest"],
#                            token_to_id["<EOS>"]])

model_input = torch.tensor([token_to_id["who"],
                            token_to_id["is"],
                            token_to_id["touati"],
                            token_to_id["kamel"],
                            token_to_id["<EOS>"]])

input_length = model_input.size(dim=0)
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 10
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))
    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))


print("Predicted Tokens:\n")
for id in predicted_ids:
  print("\t", id_to_token[id.item()])

Predicted Tokens:

	 kamel
	 kamel
	 kamel
	 kamel
	 kamel
	 kamel


In [ ]:
trainer = L.Trainer(max_epochs=20)
trainer.fit(model, train_dataloaders=dataloader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | we             | Embedding        | 16     | train
1 | pe             | PositionEncoding | 0      | train
2 | self_attention | Attention        | 12     | train
3 | fc_layer       | Linear           | 24     | train
4 | loss           | CrossEntropyLoss | 0      | train
------------------------------------------------------------
52        Trainable params
0   

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
# model_input = torch.tensor([token_to_id["what"],
#                            token_to_id["is"],
#                            token_to_id["statquest"],
#                            token_to_id["<EOS>"]])

model_input = torch.tensor([token_to_id["who"],
                            token_to_id["is"],
                            token_to_id["touati"],
                            token_to_id["kamel"],
                            token_to_id["<EOS>"]])

input_length = model_input.size(dim=0)
predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

max_length = 10
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))
    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))


print("Predicted Tokens:\n")
for id in predicted_ids:
  print("\t", id_to_token[id.item()])

Predicted Tokens:

	 i'm
	 estin
	 student
	 <EOS>
